<a href="https://colab.research.google.com/github/sharmahitesh381/Bank-Customer-Churn-Prediction/blob/main/Bank_Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATASET LOADING FROM KAGGLE

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling/data")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/churn-modelling/Churn_Modelling.csv")
df

In [ ]:
df1 = df.drop(columns =['RowNumber','CustomerId','Surname',])
df1

EDA

In [ ]:
!pip install ydata-profiling

In [7]:
from ydata_profiling import ProfileReport
prof = ProfileReport(df1)
prof.to_file(output_file='hitesh.html')

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

FEATURE ENGINEERING

In [8]:
df1.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3776,750,France,Male,33,2,152302.72,1,1,0,71333.44,0
1996,534,France,Male,62,2,0.00,2,0,0,42763.12,1
5619,785,Spain,Male,41,7,0.00,2,1,1,199108.88,0
7416,583,France,Male,36,8,0.00,2,0,1,5571.59,0
1087,498,France,Female,31,10,0.00,2,1,0,13892.57,0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline


In [10]:
X_train , X_test , y_train , y_test = train_test_split(df1.drop(columns = ['Exited'],axis =1),df1['Exited'], test_size = 0.3, random_state = 0)

In [11]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7681,641,France,Male,33,2,146193.60,2,1,1,55796.83
9031,541,France,Male,39,7,0.00,2,1,0,19823.02
3691,590,France,Female,76,5,160979.68,1,0,1,13848.58
202,516,Spain,Male,50,5,0.00,1,0,1,146145.93
5625,508,France,Female,60,7,143262.04,1,1,1,129562.74
...,...,...,...,...,...,...,...,...,...,...
9225,594,Germany,Female,32,4,120074.97,2,1,1,162961.79
4859,794,Spain,Female,22,4,114440.24,1,1,1,107753.07
3264,738,France,Male,35,5,161274.05,2,1,0,181429.87
9845,590,Spain,Female,38,9,0.00,2,1,1,148750.16


In [12]:
y_train

,Exited
7681,1
9031,0
3691,0
202,1
5625,0
...,...
9225,0
4859,0
3264,0
9845,0


In [13]:
#onehotencoding
tnf1 = ColumnTransformer([('one_hot_encode',OneHotEncoder(sparse_output =False , drop = 'first',handle_unknown='ignore'),[1,2])])

In [14]:
#feature scaling
tnf2 = ColumnTransformer([('feature_scaling', MinMaxScaler(),slice(0,10))])

In [15]:
#feature selection
tnf3 = SelectKBest(score_func=chi2,k=6)

In [16]:
#train the model
tnf4 = LogisticRegression()

PIPELINE


In [17]:
pipe = make_pipeline(tnf1,tnf2,tnf3,tnf4)

In [18]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=6 is greater than n_features=3. All the features will be returned.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(transformers=[('one_hot_encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 2])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('feature_scaling',
                                                  MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7d98510b1760>)),
                ('logisticregression', LogisticRegression())])

PREDICTION AND ACCURACY

In [19]:
y_pred = pipe.predict(X_test)

In [20]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.793

EXPORTING THE PIPELINE

In [22]:
import pickle

In [23]:
pickle.dump(pipe,open('pipe.pkl','wb'))

TESTING


In [24]:
import pickle

try:
    with open('pipe.pkl', 'rb') as f:
        loaded_pipe = pickle.load(f)
    print("Pipeline loaded successfully!")
    # You can now use 'loaded_pipe'
    # For example, to check its type or content:
    # print(type(loaded_pipe))
except EOFError:
    print("Error: The 'pipe.pkl' file is empty or corrupted. Please ensure it was created correctly by re-running the 'pickle.dump' cell with a 'with' statement.")
except Exception as e:
    print(f"An unexpected error occurred while loading the pipeline: {e}")


Pipeline loaded successfully!


In [25]:
random_data = np.array([400,'Spain','Female',40,5,10000,1,1,1,25000],dtype =object).reshape(1,10)

In [26]:
pipe.predict(random_data)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([0])